## Structured Output from your LLM (Json Persing)

In [51]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing import List

In [52]:
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

Define the output structure we want from the model

In [59]:
class Activity(BaseModel):
    
    location: str = Field(title="Location", description="Location for fun activity")
    # add a desription field for the location
    activity: List[str] = Field(title = "Activity", description="List of fun activities that one can do in the location")
    
class ListActivities(BaseModel):
    activities: List[Activity] = Field(title = "Activities", description="List of fun locations and the activities that one can do in the location")

In [60]:
# Set up a parser
parser = PydanticOutputParser(pydantic_object=ListActivities)

In [61]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Activity": {"properties": {"location": {"description": "Location for fun activity", "title": "Location", "type": "string"}, "activity": {"description": "List of fun activities that one can do in the location", "items": {"type": "string"}, "title": "Activity", "type": "array"}}, "required": ["location", "activity"], "title": "Activity", "type": "object"}}, "properties": {"activities": {"description": "List of fun locations and the activities that one can do in the location", "items": {"$ref": "#/$defs/Activity"}, "title": "A

In [62]:
# inject instructions into the prompt template.
prompt = PromptTemplate(
    template="""
    Your are a helpful travel agent AI bot and your role is to answer any tavel related question the user might ask.
    Here is the users question: {user_input}
    
    {format_instructions}
    
    """,
    input_variables=["user_input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [63]:
prompt

PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Activity": {"properties": {"location": {"description": "Location for fun activity", "title": "Location", "type": "string"}, "activity": {"description": "List of fun activities that one can do in the location", "items": {"type": "string"}, "title": "Activity", "type": "array"}}, "required": ["location", "activity"], "title": "Activity", "type": "object"}}, "properties": {"activities": {"description": "List of fun location

develope the chain and invoke

In [64]:
chain = prompt | model | parser
response = chain.invoke({"user_input":"what can i do in Tanzania?"})

In [67]:
response.activities[0]

Activity(location='Serengeti National Park', activity=['Safari game drives', 'Hot air balloon safaris', 'Cultural visits to local tribes'])

In [69]:
res_dict = response.dict()

In [75]:
res_dict["activities"][0]

{'location': 'Serengeti National Park',
 'activity': ['Safari game drives',
  'Hot air balloon safaris',
  'Cultural visits to local tribes']}